## Sending Data from a File

In [47]:
import serial
import json
#helper functions:
def getTxPower(rssi):
    if rssi > -60:
        return 84
    elif rssi > -70:
        return 44
    elif rssi > -85:
        return 8
    else:
        return 8
    
def parse_input(e):
    #find mac
    tmp = e.find('"MAC":') + len('"MAC:"')
    mac = e[tmp:tmp+17]
    #find RSSI
    tmp = e.find('"RSSI":') + len('"RSSI:"')
    rssi = e[tmp:tmp+3]
    #find SSID
    tmp = e.find('"SSID":') + len('"SSID":')
    ssid = e[tmp:-1]
    return mac, rssi, ssid

In [49]:
port_name = "/dev/ttyUSB0"

# Prepare data to send
file_path = "12-03/data4.json"
data = json.load(open(file_path, 'r'))[0]
macs = []
rssi = []
ssids = []
rssi = rssi[:15]
#convert to tx power values
tx = []
for i in range(len(rssi)):
    tx.append(getTxPower(rssi[i]))

# Open serial port
ser = serial.Serial(port_name, 115200)

# Read data from serial port
while True:
    data = ser.readline()
    data = data.decode("utf-8")

    if "Waiting for input" in data:
        print("connected")
        break
    else:    
        print(data)

#bake up the command string
print(f"sending {len(rssi)} values")
cmd = ""
for i in range(len(rssi)):
    cmd += macs[i] + ";" + str(tx[i]) + ";" + ssids[i] + ","
    print(f"{rssi[i]} -> {tx[i]}")

#send cmd to serial port
#cmd format: "mac;tx;ssid,mac;tx;ssid..."
print(cmd)
ser.write(cmd.encode("utf-8"))
print("sent")
try:
    while True:
        #if available, read data from serial port
        if ser.in_waiting > 0:
            data = ser.readline().decode("utf-8")
            print(data)
except Exception as e:
    print(e)
    ser.close()
    print("closed")


## Sending Data Manually

In [ ]:
import serial
#Manually Send data to serial port
macs = ['34:8A:12:C9:22:8','34:8A:12:C9:22:24','34:8A:12:C9:22:44','34:8A:12:C9:22:64','34:8A:12:C9:22:84']
tx = ['8','24','44','64','84']
ssids = ['test1','test2','test3','test4','test5']

#open serial port
ser = serial.Serial('/dev/ttyUSB1', 115200)

#wait for connection
while True:
    if(ser.in_waiting > 0):
        data = ser.readline().decode("utf-8")
        print(data)
        if "Waiting for input" in data:
            print("connected")
            break

#bake up the command string
cmd = ""
for i in range(len(macs)):
    cmd += macs[i] + ";" + str(tx[i]) + ";" + ssids[i] + ","

#send cmd to serial port
ser.write(cmd.encode("utf-8"))
print("sent")

try:
    while True:
        #if available, read data from serial port
        if ser.in_waiting > 0:
            data = ser.readline().decode("utf-8")
            print(data)
except:
    ser.close()
    print("closed")

## Programmable Attack

In [ ]:
import serial
import json
import time
from pandas import DataFrame

#program file = attack.json
file_path = "hyundai_raw/-1_0.json"
data = json.load(open(file_path, 'r'))

df = DataFrame()
for entry in data:
    #entry is a list of dict which is in stirng
    for e in entry:
        #find mac
        tmp = e.find('"MAC":') + len('"MAC:"')
        mac = e[tmp:tmp+17]
        #find RSSI
        tmp = e.find('"RSSI":') + len('"RSSI:"')
        rssi = int(e[tmp:tmp+3])
        #find SSID
        tmp = e.find('"SSID":') + len('"SSID":')
        ssid = e[tmp:-1]
        
        #new row for datafram
        new_row = {'MAC': mac, 'RSSI':rssi, 'SSID':ssid}
        df = df.append(new_row, ignore_index=True)
# Define aggregation functions for each column
agg_functions = {'RSSI': 'mean', 'SSID': 'first'}

# Calculate the average 'rssi' values and representative 'ssid' for each 'mac'
average_rssi_df = df.groupby('MAC').agg(agg_functions).reset_index()

# Sort the DataFrame by 'rssi'
average_rssi_df = average_rssi_df.sort_values(by='RSSI', ascending=False)
# Reset the index
average_rssi_df.reset_index(drop=True, inplace=True)
final_df = average_rssi_df.head(10)

#open serial port
ser = serial.Serial('COM3', 115200)

#wait for connection
while True:
    if(ser.in_waiting > 0):
        data = ser.readline().decode("utf-8")
        print(data)
        if "Waiting for input" in data:
            print("connected")
            break

#bake up the command string
cmd = ""
for i, row in final_df.iterrows():
    tx = str(getTxPower(row['RSSI']))
    cmd += row['MAC'] + ";" + tx + ";" + row['SSID'] + ","
ser.write(cmd.encode("utf-8"))
print("sent")
    
#read serial
try:
    while True:
        if ser.in_waiting > 0:
            data = ser.readline().decode("utf-8")
            print(data)
except:
    ser.write(bytes([0x00]))
    ser.close()
    print("closed")